In [12]:
import pickle
import numpy as np

MY_PATH = "E:/python/pythondata/StringNet/"
FILE_HEADER = "StringNet"
NODE_NAME = "test03_100_10_1"
index = 0

file_name = MY_PATH + FILE_HEADER + '_' + NODE_NAME + '_th_' + str(index)
with open(file_name, 'rb') as File:
    mylist = pickle.load(File)
    
tv1 = mylist[0]
tv2 = mylist[1]
tv3 = mylist[2]
tv4 = mylist[3]

np.mean(tv1)

AttributeError: Can't get attribute 'StringNet' on <module 'ipykernel_launcher' from 'C:\\Users\\Lenovo\\anaconda3\\lib\\site-packages\\ipykernel_launcher.py'>

In [15]:
# ==================   The StringNet class    ===========================
class StringNet:

    # Initialization of the StringNet Object
    def __init__(self, n):
        self.circle = [0 for _ in range(n)]     # type: list
        self.leg = [[]]                         # type: list
        self.pipe = [[] for _ in range(n)]      # type: list
        self.line = [[] for _ in range(n)]      # type: list
        self.vbegin = [i for i in range(n)]     # type: list
        self.circle_number = n                  # type: int
        self.stack_circle = [[], n]             # type: list
        self.stack_leg = [[], 1]                # type: list
        self.N = n                              # type: int

    def fn_print(self):
        print('============================================')
        print('circle:')
        print(self.circle)
        print('leg:')
        print(self.leg)
        print('pipe:')
        print(self.pipe)
        print('line:')
        print(self.line)
        print('vbegin:')
        print(self.vbegin)
        print('stack_circle:')
        print(self.stack_circle)
        print('stack_leg')
        print(self.stack_leg)
        print('============================================')

    # find the circle index at position (line,y)
    def fn_find_circle(self, line, y):

        # if the line is empty, fetch the circle number in "vbegin"
        if not self.line[line]:
            ind_leg_up = -1
            ind_leg_down = -1
            circle_label = self.vbegin[line]
        else:
            # fetch the y coordinate in the line-th "line"
            tv = [self.line[line][i][1] for i in range(len(self.line[line]))]
            (ind_up, ind_down) = self.fn_find_nearest(y, tv)
            if ind_up != -1:
                # fetch the index for the nearest leg
                ind_leg_up = self.line[line][ind_up][0]
                ind_leg_down = self.line[line][ind_down][0]
                circle_label = self.leg[ind_leg_up][3]

            else:
                # if failure in the call fn_find_nearest()
                ind_leg_up = -1
                ind_leg_down = -1
                circle_label = -1
        return ind_leg_up, ind_leg_down, circle_label

    # find the nearest index to the value y, in the vector vy
    @staticmethod
    def fn_find_nearest(y, vy):
        # In fact the input "vy" is sorted by descend order
        ind_large = None
        for ind_large in (x for x in range(len(vy)) if vy[x] >= y):
            break
        if not ind_large:
            ind_up = len(vy) - 1
            ind_down = 0
        elif vy[ind_large] == y:
            ind_up = -1
            ind_down = -1
        elif ind_large == 0:
            ind_up = len(vy) - 1
            ind_down = 0
        else:
            ind_up = ind_large - 1
            ind_down = ind_large

        return ind_up, ind_down

    # ==========  The functions of the class StringNet
    #               fn_stack_leg_pull()
    #               fn_stack_leg_push()
    #               fn_stack_circle_pull()
    #               fn_stack_circle_push()
    # maintain the stack for generation of the index of the legs and circles respectively.
    def fn_stack_leg_pull(self):
        if not self.stack_leg[0]:
            num_leg = self.stack_leg[1]
            self.stack_leg[1] = num_leg + 1
            self.leg.append([])
        else:
            num_leg = self.stack_leg[0][-1]
            self.stack_leg[0].pop()
        return num_leg

    def fn_stack_leg_push(self, index):
        self.stack_leg[0].append(index)

    def fn_stack_circle_pull(self):
        if not self.stack_circle[0]:
            num_circle = self.stack_circle[1]       # type: int
            self.stack_circle[1] = num_circle + 1
            self.circle.append([])
        else:
            num_circle = self.stack_circle[0][-1]   # type: int
            self.stack_circle[0].pop()
        return num_circle

    def fn_stack_circle_push(self, index):
        self.stack_circle[0].append(index)

    def fn_update_line(self, line, num_leg, y):
        ind_large = len(self.line[line])
        for ind_large in (x for x in range(len(self.line[line])) if self.line[line][x][1] > y):
            break
        self.line[line][ind_large:ind_large] = [[num_leg, y]]

    def fn_update_line2(self, line, num_leg):
        self.line[line] = [x for x in self.line[line] if x[0] != num_leg]

    def fn_update_pipe(self, pipe, num_leg, y):
        ind_large = len(self.pipe[pipe])
        for ind_large in (x for x in range(len(self.pipe[pipe])) if self.pipe[pipe][x][1] > y):
            break
        self.pipe[pipe][ind_large:ind_large] = [[num_leg, y]]

    def fn_update_pipe2(self, pipe, num_leg):
        self.pipe[pipe] = [x for x in self.pipe[pipe] if x[0] != num_leg]

    def fn_update_leg(self, circle_number):
        if self.circle[circle_number] != 0:
            for x in range(len(self.circle[circle_number])):
                if x % 2 == 0:
                    leg_number = self.circle[circle_number][x]      # type: int
                    self.leg[leg_number][2] = circle_number
                else:
                    leg_number = -self.circle[circle_number][x]   # type: int
                    self.leg[leg_number][3] = circle_number

    def fn_find_leg(self, ind_leg):
        pipe = self.leg[ind_leg][0]     # type: int
        line1 = pipe    # type: int
        # line2 = (pipe+1)%self.N
        n1 = len(self.line[line1])
        # n2 = len(self.line[line2])

        p1 = None
        for p1 in (x for x in range(n1) if self.line[line1][x][0] == ind_leg):
            break
        # for p2 in (x for x in range(n2) if self.line[line2][x][0]==ind_leg): break
        left_up = self.line[line1][(p1 - 1) % n1][0]
        left_down = self.line[line1][(p1 + 1) % n1][0]
        # right_up = self.line[line2][(p2-1)%n2][0]
        # right_down = self.line[line2][(p2+1)%n2][0]
        return left_up, left_down

    def fn_crmap2(self, dis):
        vx = [self.vbegin[k] if (not self.line[k]) else self.leg[self.line[k][0][0]][2] for k in range(self.N)]
        tv = [1 if vx[k] == vx[(k + dis) % self.N] else 0 for k in range(self.N)]
        return tv

    def fn_crmap3(self, dis):
        vx = np.array([self.vbegin[k] if (not self.line[k])
                       else self.leg[self.line[k][0][0]][2] for k in range(self.N)])
        vx2 = np.hstack((vx[dis:], vx[:dis]))

        return (vx == vx2).astype('int')

    '''
    def fn_crmap(self, dis):
        vx = np.zeros(self.N)
        for k in range(self.N):
            if not self.line[k]:
                vx[k] = self.vbegin[k]
            else:
                vx[k] = self.leg[self.line[k][0][0]][2]

            vx2 = np.append(vx[dis:], vx[:dis])

        return (vx == vx2).astype('int')
    '''

    def fn_which_circle(self):
        vx = np.zeros(self.N)
        for k in range(self.N):
            if not self.line[k]:
                vx[k] = self.vbegin[k]
            else:
                vx[k] = self.leg[self.line[k][0][0]][2]

        return vx

    def fn_sort_circle_plus(self, circle, p_plus, p_minus):
        if p_plus == 0 and p_minus == len(self.circle[circle]) - 1:
            return
        elif p_plus + 1 == p_minus:
            self.circle[circle] = self.circle[circle][p_minus:] + self.circle[circle][:p_minus]
            self.circle[circle].reverse()
        elif p_plus - 1 == p_minus:
            self.circle[circle] = self.circle[circle][p_plus:] + self.circle[circle][:p_plus]

        else:
            print('error')

    def fn_sort_circle_minus(self, circle, p_plus, p_minus):
        if p_plus == 0 and p_minus == len(self.circle[circle]) - 1:
            self.circle[circle].reverse()
        elif p_plus + 1 == p_minus:
            self.circle[circle] = self.circle[circle][p_minus:] + self.circle[circle][:p_minus]
        elif p_plus - 1 == p_minus:
            self.circle[circle] = self.circle[circle][p_plus:] + self.circle[circle][:p_plus]
            self.circle[circle].reverse()
        else:
            print('error')

    # Insert the leg
    def fn_insert_leg(self, pipe, y):
        # line1 and line2 are lines of the left-side and right-side of the pipe
        # respectively
        line1 = pipe
        line2 = (pipe + 1) % self.N
        (leg_up_left, leg_down_left, circle_left) = self.fn_find_circle(line1, y)
        (leg_up_right, leg_down_right, circle_right) = self.fn_find_circle(line2, y)

        # res = -1 means the failure of the insertion
        if circle_left == -1 or circle_right == -1:
            # res = -1
            print('return')
            return -1
        # else:
        #     res = 1

        # pull a index "num_leg" for the new generated leg
        num_leg = self.fn_stack_leg_pull()
        if num_leg == len(self.leg):
            self.leg.append([pipe, y, -1, -1])
        else:
            self.leg[num_leg] = [pipe, y, -1, -1]

        # left circle and right circle are all empty circle
        if (self.circle[circle_left] == 0) and (self.circle[circle_right] == 0):
            num1 = min([circle_left, circle_right])
            num2 = max([circle_left, circle_right])
            self.circle[num1] = [num_leg, -num_leg]
            self.circle[num2] = []
            self.fn_stack_circle_push(num2)  # the circle[num2] is deleted
            self.fn_update_leg(num1)
            self.circle_number = self.circle_number - 1
        # left empty but right not
        elif (self.circle[circle_left] == 0) and (not self.circle[circle_right] == 0):
            point_plus = self.circle[circle_right].index(leg_down_right)
            point_minus = self.circle[circle_right].index(-leg_up_right)
            if point_plus + 1 == point_minus:
                self.circle[circle_right][point_minus:point_minus] = [-num_leg, num_leg]
            elif point_plus - 1 == point_minus:
                self.circle[circle_right][point_plus:point_plus] = [num_leg, -num_leg]
            else:
                self.circle[circle_right][point_minus + 1:point_minus + 1] = [num_leg, -num_leg]
            self.circle[circle_left] = []
            self.fn_stack_circle_push(circle_left)
            self.fn_update_leg(circle_right)
            self.circle_number = self.circle_number - 1
            # right empty but left not
        elif (not self.circle[circle_left] == 0) and (self.circle[circle_right] == 0):
            point_plus = self.circle[circle_left].index(leg_down_left)
            point_minus = self.circle[circle_left].index(-leg_up_left)
            if point_plus + 1 == point_minus:
                self.circle[circle_left][point_minus:point_minus] = [-num_leg, num_leg]
            elif point_plus - 1 == point_minus:
                self.circle[circle_left][point_plus:point_plus] = [num_leg, -num_leg]
            else:
                self.circle[circle_left][point_minus + 1:point_minus + 1] = [num_leg, -num_leg]
            self.circle[circle_right] = []
            self.fn_stack_circle_push(circle_right)
            self.fn_update_leg(circle_left)
            self.circle_number = self.circle_number - 1
        else:  # if the left and the right circles are not empty

            # if the left and right circles are different
            if circle_left != circle_right:

                if len(self.circle[circle_left]) < len(self.circle[circle_right]):
                    num1 = circle_right
                    num2 = circle_left
                    p1_plus = self.circle[num1].index(leg_down_right)
                    p1_minus = self.circle[num1].index(-leg_up_right)
                    p2_plus = self.circle[num2].index(leg_down_left)
                    p2_minus = self.circle[num2].index(-leg_up_left)
                else:
                    num1 = circle_left
                    num2 = circle_right
                    p1_plus = self.circle[num1].index(leg_down_left)
                    p1_minus = self.circle[num1].index(-leg_up_left)
                    p2_plus = self.circle[num2].index(leg_down_right)
                    p2_minus = self.circle[num2].index(-leg_up_right)

                self.fn_sort_circle_plus(num1, p1_plus, p1_minus)
                self.fn_sort_circle_minus(num2, p2_plus, p2_minus)
                self.circle[num1] = self.circle[num1] + [num_leg] + self.circle[num2] + [-num_leg]
                self.circle[num2] = []
                self.circle_number = self.circle_number - 1
                self.fn_stack_circle_push(num2)
                self.fn_update_leg(num1)

            else:  # if the left circle and the right circle are the same

                # need to separate one circle, so we need a new circle index
                num_circle = self.fn_stack_circle_pull()
                self.circle_number = self.circle_number + 1
                if leg_up_left == leg_up_right and leg_down_left == leg_down_right:
                    self.circle[circle_left] = [num_leg, -leg_up_left]
                    self.circle[num_circle] = [leg_down_left, -num_leg]
                elif leg_up_left == leg_up_right and leg_down_left != leg_down_right:
                    find_ind = self.circle[circle_left].index(-leg_up_left)
                    self.circle[circle_left][find_ind] = -num_leg
                    self.circle[num_circle] = [num_leg, -leg_up_left]
                elif leg_up_left != leg_up_right and leg_down_left == leg_down_right:
                    find_ind = self.circle[circle_left].index(leg_down_left)
                    self.circle[circle_left][find_ind] = num_leg
                    self.circle[num_circle] = [leg_down_left, -num_leg]
                else:
                    # the big circle
                    p1_plus = self.circle[circle_left].index(leg_down_left)
                    p1_minus = self.circle[circle_left].index(-leg_up_left)
                    p2_plus = self.circle[circle_left].index(leg_down_right)
                    p2_minus = self.circle[circle_left].index(-leg_up_right)
                    if p1_plus + 1 == p1_minus:
                        if p1_minus < p2_minus < p2_plus:
                            self.circle[num_circle] = [num_leg] + self.circle[circle_left][p1_minus:p2_minus + 1]
                            self.circle[circle_left] = self.circle[circle_left][p2_plus:] \
                                + self.circle[circle_left][:p1_plus + 1] + [-num_leg]

                        elif p1_minus > p2_minus:
                            self.circle[num_circle] = self.circle[circle_left][p2_plus:p1_plus + 1] + [-num_leg]
                            self.circle[circle_left] = [num_leg] + self.circle[circle_left][p1_minus:] \
                                + self.circle[circle_left][:p2_minus + 1]

                        else:
                            self.circle[num_circle] = [num_leg] + self.circle[circle_left][p1_minus:p2_minus + 1]
                            self.circle[circle_left] = self.circle[circle_left][p2_plus:p1_plus + 1] + [-num_leg]

                    elif p1_plus - 1 == p1_minus:
                        if p1_plus < p2_plus:
                            self.circle[num_circle] = [num_leg] + self.circle[circle_left][p2_minus:] \
                                                      + self.circle[circle_left][:p1_minus + 1]
                            self.circle[circle_left] = self.circle[circle_left][p1_plus:p2_plus + 1] + [-num_leg]

                        else:
                            self.circle[num_circle] = [num_leg] + self.circle[circle_left][p2_minus:p1_minus + 1]
                            self.circle[circle_left] = self.circle[circle_left][p1_plus:] \
                                + self.circle[circle_left][:p2_plus + 1] + [-num_leg]

                    else:
                        self.circle[num_circle] = [num_leg] + self.circle[circle_left][p2_minus:p1_minus + 1]
                        self.circle[circle_left] = self.circle[circle_left][p1_plus:p2_plus + 1] + [-num_leg]

                self.fn_update_leg(circle_left)
                self.fn_update_leg(num_circle)

        self.fn_update_line(line1, num_leg, y)
        self.fn_update_line(line2, num_leg, y)
        self.fn_update_pipe(pipe, num_leg, y)

        return num_leg

    # Delete the leg
    def fn_delete_leg(self, ind_leg):
        circle_up = self.leg[ind_leg][2]            # type: int
        circle_down = self.leg[ind_leg][3]          # type: int
        pipe = self.leg[ind_leg][0]                 # type: int
        y = self.leg[ind_leg][1]                    # type: float
        line1 = pipe                                # type: int
        line2 = (pipe + 1) % self.N                 # type: int

        if circle_up == circle_down:

            new_circle = self.fn_stack_circle_pull()            # type: int
            self.circle_number = self.circle_number + 1
            tv = self.circle[circle_up]
            length_circle = len(tv)
            if length_circle == 2:
                self.circle[circle_up] = 0
                self.circle[new_circle] = 0
                self.vbegin[line1] = circle_up
                self.vbegin[line2] = new_circle
            else:
                p_plus = tv.index(ind_leg)
                p_minus = tv.index(-ind_leg)
                if p_minus + 1 == p_plus:
                    p_add = p_plus + 1
                    leg_next = -tv[p_add]
                    line1_next = self.leg[leg_next][0]              # type: int
                    self.circle[new_circle] = 0
                    self.circle[circle_up][p_minus:p_plus + 1] = []
                    if (line1_next - line1) % self.N == 1:
                        self.vbegin[line1] = new_circle
                    elif (line1_next - line1) % self.N == self.N - 1:
                        self.vbegin[line2] = new_circle
                    else:
                        print('error')
                        return
                elif p_plus + 1 == p_minus:
                    p_add = (p_minus + 1) % length_circle
                    leg_next = tv[p_add]
                    line1_next = self.leg[leg_next][0]
                    self.circle[new_circle] = 0
                    self.circle[circle_up][p_plus:p_minus + 1] = []
                    if (line1_next - line1) % self.N == 1:
                        self.vbegin[line1] = new_circle
                    elif (line1_next - line1) % self.N == self.N - 1:
                        self.vbegin[line2] = new_circle
                    else:
                        print('error')
                        return
                elif p_plus == 0 and p_minus == length_circle - 1:
                    p_add = 1
                    leg_next = -tv[p_add]
                    line1_next = self.leg[leg_next][0]
                    self.circle[new_circle] = 0
                    self.circle[circle_up] = self.circle[circle_up][2:-1] + [self.circle[circle_up][1]]
                    if (line1_next - line1) % self.N == 1:
                        self.vbegin[line1] = new_circle
                    elif (line1_next - line1) % self.N == self.N - 1:
                        self.vbegin[line2] = new_circle
                    else:
                        print('error')
                        return
                else:
                    if p_minus < p_plus:
                        self.circle[new_circle] = (tv[p_plus + 1:] + tv[:p_minus])
                        self.circle[new_circle].reverse()
                        self.circle[circle_up] = tv[p_minus + 1:p_plus]
                    else:
                        self.circle[new_circle] = (tv[p_plus + 1:p_minus])
                        self.circle[new_circle].reverse()
                        self.circle[circle_up] = tv[p_minus + 1:] + tv[:p_plus]
            self.leg[ind_leg] = []
            self.fn_update_leg(circle_up)
            self.fn_update_leg(new_circle)

        else:
            (left_up, left_down) = self.fn_find_leg(ind_leg)
            # print([left_up,right_up,left_down,right_down])
            v_upc = self.circle[circle_up]
            v_dnc = self.circle[circle_down]
            p1_plus = v_upc.index(ind_leg)
            p1_minus = v_upc.index(-left_up)
            p2_minus = v_dnc.index(-ind_leg)
            p2_plus = v_dnc.index(left_down)

            # print([p1_plus,p1_minus,p2_plus,p2_minus])
            if p1_minus < p1_plus or (p1_minus == len(v_upc) - 1 and p1_plus == 0):
                order1 = True
            else:
                order1 = False
            if p2_minus < p2_plus or (p2_minus == len(v_dnc) - 1 and p2_plus == 0):
                order2 = False
            else:
                order2 = True

            tv1 = v_upc[p1_plus + 1:] + v_upc[:p1_plus]
            # tv2 = v_dnc[p2_minus+1:] + v_dnc[:p2_minus]

            if (order1 and (not order2)) or ((not order1) and order2):
                self.circle[circle_down][p2_minus:p2_minus + 1] = tv1
            else:
                tv1.reverse()
                self.circle[circle_down][p2_minus:p2_minus + 1] = tv1

            self.circle[circle_up] = []
            self.circle_number = self.circle_number - 1
            self.fn_stack_circle_push(circle_up)
            self.leg[ind_leg] = []
            self.fn_update_leg(circle_down)

        self.fn_update_line2(line1, ind_leg)
        self.fn_update_line2(line2, ind_leg)
        self.fn_update_pipe2(pipe, ind_leg)
        self.fn_stack_leg_push(ind_leg)

        return pipe, y

# ==============  End StringNet class  ========================================

In [16]:
import pickle
import numpy as np

MY_PATH = "E:/python/pythondata/StringNet/"
FILE_HEADER = "StringNet"
NODE_NAME = "test03_100_10_1"
index = 0

file_name = MY_PATH + FILE_HEADER + '_' + NODE_NAME + '_th_' + str(index)
with open(file_name, 'rb') as File:
    mylist = pickle.load(File)
    
tv1 = mylist[0]
tv2 = mylist[1]
tv3 = mylist[2]
tv4 = mylist[3]

np.mean(tv1)

AttributeError: Can't get attribute 'StringNet' on <module 'ipykernel_launcher' from 'C:\\Users\\Lenovo\\anaconda3\\lib\\site-packages\\ipykernel_launcher.py'>

In [5]:
tv3

(1000, 1000, 1, 0)

In [6]:
len(tv1)

1000000

In [7]:
len(tv2)

1

In [8]:
tv2

[374117]

In [9]:
len(tv1)

1000000

In [10]:
type(tv1)

numpy.ndarray